In [1]:
import os, nltk, re
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import NLTKTextSplitter,RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains.summarize import load_summarize_chain
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from pathlib import Path as p
os.environ["OPENAI_API_KEY"]=open(r"C:\Users\shaur\Downloads\chat_gpt_key.txt",'r').read()
os.chdir(r"C:\Users\shaur\Desktop\icici policies wording")
os.getcwd()

C:\Users\shaur\AppData\Roaming\Python\Python39\site-packages\langchain_community\document_loaders\blob_loaders\file_system.py:5: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_community.document_loaders.blob_loaders.schema import Blob, BlobLoader
C:\Users\shaur\AppData\Roaming\Python\Python39\site-packages\langchain_community\document_loaders\__init__.py:221: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic

'C:\\Users\\shaur\\Desktop\\icici policies wording'

In [2]:
all_policies_pdfs = [i for i in os.listdir() if '.pdf' in i]
all_policies_dict = {}
for i in range(0,len(all_policies_pdfs)):
    loader = PyPDFLoader(all_policies_pdfs[i])
    pages = loader.load()
    pages = [page.page_content for page in pages]
    print(f"Length of pages from {all_policies_pdfs[i]}:- {len(pages)}")
    all_policies_dict[all_policies_pdfs[i]] = pages
all_policies_dict.keys()

Length of pages from Elevate Policy Wordings.pdf:- 45
Length of pages from Health AdvantEdge_Policy wordings_IL.pdf:- 51
Length of pages from policy-wordings_maxprotect.pdf:- 40


dict_keys(['Elevate Policy Wordings.pdf', 'Health AdvantEdge_Policy wordings_IL.pdf', 'policy-wordings_maxprotect.pdf'])

In [3]:
# all_policies_dict['Elevate Policy Wordings.pdf']

In [4]:
# all_policies_dict['Health AdvantEdge_Policy wordings_IL.pdf']

In [5]:
# all_policies_dict['policy-wordings_maxprotect.pdf']

In [6]:
repeted_text_0 = 'IRDA Reg. No. 115\nMailing Address:\n601 / 602, 6th Floor, Interface Building No. 16,\nNew Link Road, Malad (West), \nMumbai - 400 064.CIN:  L67200MH2000PLC129408\nRegistered Office Address:\nICICI Lombard House, 414, P Balu Marg, Off \nVeer Savarkar Road, Nr Siddhi Vinayak Temple, \nPrabhadevi, Mumbai - 400 025.UIN: ICIHLIP25048V042425 Product Name: Elevate\nToll free No.  : 1800 2666  \nAlternate No.: 86552 22666 (Chargeable)\nWebsite :  www.iciclombard.com\nE-mail : customersupport@icicilombard.comICICI Lombard General Insurance Company Limited'
repeted_text_1 = 'Health AdvantEdge  \nICICI Lombard General Insurance Company Limited  \n       IRDA  Reg. No. 115                            CIN : L67200MH2000PLC129408                                  UIN: ICIHLIP24182V042324     Health A dvantEdge  \n       Mailing A ddress:                              Registered Office A ddress:                                Toll free no : 1800 2666   \n601 & 602, 6th Floor, Interf ace 16,    ICICI Lombard House, 414, P Balu Marg,                     A lternate no :  86552 2 2666 (chargeable)   \nNew Linking Road, Malad (West)      Of f  Veer Sav arkar Road, Nr Siddhi Vinay ak Temple,    E-mail : customersupport@icicilombard.com  \n    Mumbai - 400 064                             Prabhadev i, Mumbai 400 025                                        Website : www.icicilombard.com   '
repeted_text_2 = 'Annexure I  MaxProtect  Policy w ordings  \nIRDAI reg. no.: 115                CIN: L67200MH20000PLC129408   UIN: ICIHLIP24084V012324'
for i in all_policies_dict.keys():
    for k in [repeted_text_0,repeted_text_1,repeted_text_2]:
        all_policies_dict[i] = [page.replace(k.strip(), '').strip() for page in all_policies_dict[i]]

In [7]:
# splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1500,
#     chunk_overlap=200,
#     separators=["\n\n", "\n", ".", " "]
# )
splitter = NLTKTextSplitter(chunk_size=1200, chunk_overlap=200)
chunked_policies = {}
for i, full_text in all_policies_dict.items():
    full_text = "\n".join(pages)
    docs = splitter.create_documents([full_text])
    chunked_policies[i] = docs

Created a chunk of size 1338, which is longer than the specified 1200
Created a chunk of size 1338, which is longer than the specified 1200
Created a chunk of size 1338, which is longer than the specified 1200


In [8]:
print(chunked_policies['policy-wordings_maxprotect.pdf'][6].page_content)

A co-payment does not reduce the Sum Insured.

Cumulative Bonus  means any increase or a ddition in the Sum Insured granted by the insurer 
without an associated increase in premium  
 
Day Care Centre  means any institution established for day care treatment of illness and/or 
injuries or a medical setup with a hospital and which has been registere d with the local 
Annexure I  MaxProtect  Policy w ordings  
IRDAI reg.

no.

: 115                CIN: L67200MH20000PLC129408   UIN: ICIHLIP24084V012324  authorities, wherever applicable, and is under supervision of a registered and qualified medical 
practitioner AND must comply with all minimum criterion as under – 
i. has qualified nursing staff under its employment;  
ii.

has qualified medical practitioner/s in charge  
iii.

has fully equipped operation theatre of its own where surgical procedures are carried 
out; 
iv.

maintains daily records of patients and will make these accessible to the insurance 
company’s authorized personne

In [9]:
chunked_policies.keys()

dict_keys(['Elevate Policy Wordings.pdf', 'Health AdvantEdge_Policy wordings_IL.pdf', 'policy-wordings_maxprotect.pdf'])

In [10]:
for i in chunked_policies.keys():
    print(f"{i}:- {len(chunked_policies[i])}")

Elevate Policy Wordings.pdf:- 120
Health AdvantEdge_Policy wordings_IL.pdf:- 120
policy-wordings_maxprotect.pdf:- 120


In [11]:
# Making an instance just to count numbers of tokens and use summarize to make summary of all policies
# Temperature 0 is for the reason that we do not want it to be creative
llm = ChatOpenAI(temperature=0,model="gpt-3.5-turbo")
total=0
for i in chunked_policies.keys():
    for k in range(0,len(chunked_policies[i])):
        total+=llm.get_num_tokens(chunked_policies[i][k].page_content)
print(f"Total Tokens to be used:- {total}")

Total Tokens to be used:- 87690


In [12]:
# the chain type have 3 type 
# map_reduce (each chuck is summarised and Combines all partial summaries into one final summary), 
# refine (keeps on refining when new chunck is added, much detailed), 
# stuff (load all chucks at onces and used as single prompt to summarize)

# summary_chain = load_summarize_chain(llm, chain_type="refine")
# final_results = {}
# for policy_name, docs in chunked_policies.items():
#     print(f"Summarizing policy: {policy_name}")
#     try:
#         map_reduce_outputs = summary_chain({"input_documents": docs})
#         final_summary = map_reduce_outputs["output_text"]
        
#         summary_dir = "policy_summaries"
#         os.makedirs(summary_dir, exist_ok=True)
#         clean_filename = policy_name.replace('.pdf', '').replace(' ', '_')
#         output_path = os.path.join(summary_dir, f"{clean_filename}_summary.txt")
#         with open(output_path, 'w', encoding='utf-8') as f:
#             f.write(final_summary)
        
#         print(f"Summary for {policy_name} saved to {output_path}")
#     except Exception as e:
#         print(f"Error processing {policy_name}: {e}")
        
# print("All policy summaries completed!")

In [29]:
#creating embedding for all documents
embeddings = OpenAIEmbeddings()
all_docs_list = []
for policy_name, docs in chunked_policies.items():
    for doc in docs:
        doc.metadata['source_policy'] = policy_name
        all_docs_list.append(doc)
#Creating local vector db
persist_directory = 'db_chroma'
vectorstore = Chroma.from_documents(
    documents=all_docs_list,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [56]:
# K = number of top k relevent chuncks
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
prompt_template = """You are a professional assistant who has to support the users to get the best Insurance plan.
The Name of 3 policy are Elevate Policy Wordings, Health AdvantEdge_Policy wordings_IL and Maxprotect.
You have to use the following pieces of context derived from 3 insurance policy documents to answer the question at the end. the User does not know anything about the policy.
If you don't know the answer based on the provided context, just say that you don't know, don't try to make up an answer.
Provide a concise answer based *only* on the context.

While suggusting insurance plan these details from the user can assist you to guide them better, ask if required. the details are as follow.
1. is the policy has to be bought for Self, Wife, Son, Daughter, Father, Mother or someone else?
2. Gender and Age
3. Health information like (medical history, including any pre-existing conditions, medications you are taking, and any surgeries or treatments)
4. Lifestyle information like (life style habits, such as smoking or drinking, as well as any risky hobbies or activities you participate)
5. Occupation information like (information about occupation, including your job title, income, and any workplace hazards you may face)

User history of the conversation is: {user_history}
Assistant history of the conversation is: {assistant_history}
If no conversation is found in History this conversation is fresh and greet the User.
If conversation is found the no need to greet agian.

New Context:{context}
New Question: {question}

Answer:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["user_history","assistant_history","context", "question"]
)
# Setting up RetrievalQA to retive similar chuncks
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

chat_logs = {"User":"","Assistant":""}

while True:
    Query = input("User:- ")
    if Query.lower() == "quit":
        break
    else:
        docs = retriever.get_relevant_documents(Query)
        full_prompt = PROMPT.format(user_history=chat_logs['User'],
                assistant_history=chat_logs['Assistant'],
                context="\n\n".join(doc.page_content for doc in docs),
                question=Query)
        response = llm.invoke(full_prompt)
        print("User:- ",Query)
        print("Assistant:- ",response.content)
        chat_logs['User'] = Query
        chat_logs['Assistant'] = response.content

User:-  Hi
Assistant:-  Hello! How can I assist you today?
User:-  I am looking out for an Insurance plan
Assistant:-  Based on the context provided, I recommend looking into the Elevate Policy Wordings, Health AdvantEdge_Policy wordings_IL, and Maxprotect insurance plans for your needs.
User:-  sure help me get the best plan for myself
Assistant:-  Based on the context provided, I recommend looking into the Elevate Policy Wordings, Health AdvantEdge_Policy wordings_IL, and Maxprotect insurance plans for your needs.
User:-  okay, suggust which one exactly?
Assistant:-  Based on the context provided, I recommend the MaxProtect insurance plan for your needs.
User:-  why? MaxProtect?
Assistant:-  Based on the context provided, the MaxProtect insurance plan offers assistance services for medical transportation and health facilitation, but the insured person is responsible for bearing the charges associated with actual services.
